# Data Pipeline and Spatial Integration

This notebook creates the base amenity dataset used in the project. It loads business licenses, food inspections, liquor licenses, and building permits from the Chicago Data Portal and processes them into a single cleaned table. Each dataset is filtered, cleaned, and converted to a geographic form with point coordinates.

The notebook performs a spatial join to assign every venue or permit to a Chicago Community Area. After joining, it aggregates counts by area and produces one output file with all amenity variables. This file captures the current supply of restaurants, bars, nightlife venues, and signals of new openings. It serves as the core input for the demographic merge and the saturation index.


In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

In [2]:

# Paths or direct URLs from Chicago Data Portal
business_licenses_url = "https://data.cityofchicago.org/resource/r5kz-chrr.csv"
food_inspections_url = "https://data.cityofchicago.org/resource/4ijn-s7e5.csv"
liquor_licenses_url = "https://data.cityofchicago.org/resource/nrmj-3kcf.csv"
building_permits_url = "https://data.cityofchicago.org/resource/ydr8-5enu.csv"
ca_boundaries_url = "https://data.cityofchicago.org/resource/igwz-8jzy.geojson"


In [3]:
# Helper to convert lat lon to geometry
def to_geodf(df, lat_col="latitude", lon_col="longitude", crs="EPSG:4326"):
    df = df.dropna(subset=[lat_col, lon_col])
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df[lon_col].astype(float), df[lat_col].astype(float)),
        crs=crs
    )
    return gdf

In [4]:
# Load Community Area boundaries
ca = gpd.read_file(ca_boundaries_url)
ca = ca.to_crs("EPSG:4326")
ca = ca.rename(columns={"area_numbe":"ca_num", "community":"ca_name"})


# 1. Business Licenses (bars, restaurants, gyms proxies)
bl = pd.read_csv(business_licenses_url)
bl["license_start_date"] = pd.to_datetime(bl["license_start_date"], errors="coerce")
bl = bl[bl["latitude"].notna() & bl["longitude"].notna()]

# Filter to relevant license categories, adjust as needed
target_categories = [
    "Retail Food Establishment",
    "Consumption on Premises",
    "Tavern",
    "Public Place of Amusement",
    "Limited Business License"
]

bl = bl[bl["license_description"].isin(target_categories)]
bl = to_geodf(bl)

In [5]:
# 2. Food Inspections
fi = pd.read_csv(food_inspections_url)
fi["inspection_date"] = pd.to_datetime(fi["inspection_date"], errors="coerce")
fi = fi[fi["latitude"].notna() & fi["longitude"].notna()]
fi = to_geodf(fi)


In [6]:
# 3. Liquor and Public Places of Amusement Licenses
liq = pd.read_csv(liquor_licenses_url)
liq = liq[liq["latitude"].notna() & liq["longitude"].notna()]
liq = to_geodf(liq)

In [7]:
# 4. Building Permits (pipeline)
bp = pd.read_csv(building_permits_url)
bp["issue_date"] = pd.to_datetime(bp["issue_date"], errors="coerce")
bp = bp[bp["latitude"].notna() & bp["longitude"].notna()]
bp = to_geodf(bp)

In [8]:

# Spatial joins
bl_join = gpd.sjoin(bl, ca, how="inner", predicate="within")
fi_join = gpd.sjoin(fi, ca, how="inner", predicate="within")
liq_join = gpd.sjoin(liq, ca, how="inner", predicate="within")
bp_join = gpd.sjoin(bp, ca, how="inner", predicate="within")

# Counts per CA
bl_counts = bl_join.groupby("ca_num").size().reset_index(name="business_license_count")
fi_counts = fi_join.groupby("ca_num").size().reset_index(name="food_inspections_count")
liq_counts = liq_join.groupby("ca_num").size().reset_index(name="liquor_license_count")
bp_counts = bp_join.groupby("ca_num").size().reset_index(name="building_permits_count")

# Merge counts
amenities = ca[["ca_num","ca_name"]].copy()
amenities = amenities.merge(bl_counts, on="ca_num", how="left")
amenities = amenities.merge(fi_counts, on="ca_num", how="left")
amenities = amenities.merge(liq_counts, on="ca_num", how="left")
amenities = amenities.merge(bp_counts, on="ca_num", how="left")

amenities = amenities.fillna(0)

# Save output
amenities.to_csv("amenities_by_CA.csv", index=False)

amenities.head()

,ca_num,ca_name,business_license_count,food_inspections_count,liquor_license_count,building_permits_count
0,1,ROGERS PARK,2.0,5.0,17.0,9.0
1,2,WEST RIDGE,16.0,11.0,12.0,6.0
2,3,UPTOWN,6.0,9.0,15.0,14.0
3,4,LINCOLN SQUARE,11.0,17.0,15.0,13.0
4,5,NORTH CENTER,4.0,7.0,22.0,18.0
